# Unix Shell (Bash)

<!-- requirement: images/.hidden_image.jpg -->
<!-- requirement: images/htop.png -->

The overwhelming majority of software development is conducted in Unix-like operating systems. The most common Unix-like operating systems today are Linux and Mac OS. Windows is not Unix-like, although such an environment is available on Windows systems via Cygwin or (as of Windows 10) the Windows Subsystem for Linux. While graphical user interfaces (GUIs) exist for modern operating systems, many software development (and data manipulation) tasks involve complex, ad-hoc operations on the file system. Such operations are typically difficult or impossible to accomplish through the GUI. Instead developers tend to interact with the operating system using a terminal.

In this notebook we'll review the basic structure of a Unix-like system as well as some of the more common file operations.

## Unix Architecture


A Unix-like operating system can be thought of as a series of layers.

|              |
|:------------:|
| Applications |
|    Shell     |
|    Kernel    |
|   Hardware   |

At the heart of the system is the computer hardware itself, such as the CPU, RAM, storage, etc. These bits of hardware are controlled by the operating system kernel. The kernel handles low-level tasks such as I/O streams, resource management, scheduling, etc.

The shell is the first layer at which users can interact. The shell is a special program which acts as an interpreter for user commands, which are sent to the kernel for execution. The shell also returns any output to the user. The overwhelmingly most commonly used shell in Linux and Mac OS is Bash. Different shells may have slightly different features and conventions for user commands, but they generally all serve the purpose of abstracting kernel operations via higher level user commands.

At the outermost layer are the applications, which might be run via GUI or the shell. The terminal itself is an application that provides access to the shell. Using a terminal we can run other applications by issuing commands to the shell.

Let's consider some of the basic applications available for use with the shell for working with the file system.

## Navigating the File System


The Unix file system is a hierarchical tree. It's root is denoted by a single forward slash, `/`.  Many system files are stored in directories high up this hierarchy, such as `/etc`, `/bin`, and `/var`. However, often a more useful reference point for navigation is our home directory, usually located at `/home/<username>` (where `<username>` is substituted by an actual username, in our case `jovyan`). Let's explore the file system.

Where are we?

In [ ]:
pwd

What's around us?

In [ ]:
ls

How do we go somewhere?

In [ ]:
cd /home/jovyan/
pwd

Notice that to navigate, we specified a path starting at the root. A path starting at root is called an *absolute path*. It is also possible to use a *relative path*.

In [ ]:
pwd
ls
cd datacourse
pwd

We started in `/home/jovyan`, under which there is a `datacourse` directory. Therefore we can specify the path to `datacourse` relative to where we are now by simply writing `datacourse`.

### File System Shortcuts


Bash has a few shortcuts for referring to common locations. The home directory is denoted by `~`, the current directory is denoted by `.`, and one directory up the hierarchy is denoted by `..`.

In [ ]:
cd ~
pwd
ls ./datacourse
ls ..

## Basic File System Operations


Often we will want to alter the content of our file system by creating, altering, or removing files. For more complex interactive tasks we might want to use a text editor (e.g. for coding) or a GUI application (e.g. for image manipulation). For simpler tasks, we can make changes to the file system directly using Bash.

To make a new directory:

In [ ]:
ls
mkdir tmp
ls

To make an (empty) file:

In [ ]:
touch tmp/tmp_file

We can also remove files and directories.

In [ ]:
rm tmp/tmp_file
rmdir tmp

Note there are two separate commands for removing files and directories. Notably, `rmdir` also requires that the directory be empty.

In [ ]:
# uncommment to run

# mkdir tmp
# touch tmp/tmp_file
# rmdir tmp

Therefore `rmdir` is used relatively little in practice. Instead we will typically use `rm` with the `-r` option. The `-r` stands for *recursive*, meaning `rm` will remove everything along the branch of the file system we specify.

In [ ]:
mkdir -p ~/tmp/a/b/c
touch ~/tmp/a/b/another_file
rm -r ~/tmp
ls ~

Most commands will take options. In the previous block we used `mkdir -p`, which creates all parent directories along the specified file path if they don't already exist.

*NOTE*: Be extremely careful with `rm`. You cannot undo `rm`. If you delete something it is gone forever. Therefore, it is often a good idea when using `rm` to first try the same command with `ls` replacing `rm`. This will show you exactly what files will be removed by `rm`.

We can additionally move (`mv`) and copy (`cp`) files from one location to another.

In [ ]:
touch mv_example
mkdir mv_dir
mv mv_example mv_dir
ls mv_dir
echo ----
ls

In [ ]:
touch cp_example
mkdir cp_dir
cp cp_example cp_dir
ls cp_dir
echo ----
ls

## Bash Command Syntax


Bash commands may invoke applications with many components. For instance the `conda` program manages Python programs as well as Python environments. Therefore sometimes we might use `conda install` while other times we might use `conda env update`, depending on what we want to do. As we've see commands additionally may take options as well as arguments. The general syntax is usually `<command(s)> <option(s)> <argument(s)>`.

For example, in some of the miniprojects we make use of the `aws` application for downloading data from AWS S3.

|command|command|command|option|argument|argument|
|:-----:|:-----:|:-----:|:----:|:------:|:------:|
|aws|s3|cp|--recursive|s3://dataincubator-course/mrdata/simple|data/simple|

## Unix `man` Pages


Now that we've learned a few Bash commands and discovered that they have additional functionality exposed by a variety of options, where could we find documentation detailing all the possible options? Many Bash commands will have a `man` (i.e. manual) page.

In [ ]:
man rm

## Search


We may want to find some particular files in our file system that we've lost. In this case we will want to use `find`.

In [ ]:
find . -name IW_Classes.ipynb

I have hidden a humorous image on your machine. Can you find it?

BONUS: Once you've found the image, try using `ls` on the directory that contains it. Do you notice anything interesting?

Sometimes we won't know what file we're looking for, but we have some idea of its contents. This is very common when working with a codebase, particularly one you didn't write yourself. For example, maybe I need to find an example of how to parse XML, and I don't remember which notebooks I might want to look at. For this we'll use `grep`, which uses regular expressions to search the content of text files.

In [ ]:
grep -r 'lxml' .

## Redirecting I/O


When working with Bash, input is read over STDIN and output is printed to STDOUT. We can think of these as channels that move information between the terminal and the kernel. By default STDIN reads from what is typed into the terminal and STDOUT prints to the terminal. However, we can redirect STDIN and STDOUT to files.

In [ ]:
ls > directory_contents
head directory_contents

Often we will want to hook up the STDIN of a command to the STDOUT of another command. This can be done with pipes. For example if I wanted to see a list of all the unique notebook prefixes on my machine.

In [ ]:
find . -name '*.ipynb' | grep -oP '[^\w]*\K([A-Z]+)(?=_)' | sort --unique

Here `find` takes care of finding all the notebooks. The output of `find` is piped into the input of `grep`, which matches the notebook prefixes. The output of `grep` is piped into the input of `sort`, which sorts the unique instances of each prefix.

## Basic Process Management


Sometimes an application may run for a long time. For instance we may ask `find` to search our entire file system, or we might simply have a text editor that remains open as we work on code. When an application is launched from the terminal, that process occupies the terminal's focus until they complete. This would compel us to open a terminal for each process we want to maintain. Fortunately, we can run processes in the background, allowing us to continue to interact with the terminal even as background processes run.

Processes can be forced to operate in the background by appending `&` to the end of the command. For example `find / -name "*.ipynb" &`. We can additionally pause a process by using the hot key `Ctrl+z`. We can then send it to the background by issuing the command `bg`.

Processes in the background can be listed using the `jobs` command. You can foreground a process by looking up its number in the `jobs` list, and entering `fg <job number>`.

To permanently interrupt a foreground process we can use the hot key `Ctrl+c`.

After we start launching all these processes, we need some way to monitor them and even destroy them if they are not doing what we want.  One tool available to us is the `ps` tool which can be invoked as `ps aux` to get all the processes.

In [ ]:
ps aux

This is a bit of a long list, luckily, we can use the magic of pipes and grep.  Lets look for only those processes which have python in their description

In [ ]:
ps aux | grep python

Much better!  If we want a more interactive view of our process, including memory and CPU usage statistics, we can use either `top` or an enhanced version `htop`.  `top` is installed on your JupyterHub pods, `htop` will need to be installed by running `sudo apt-get install htop`.  The screen will look like below:
![`htop`](images/htop.png)


Here we can see a few things:

* CPU usage of all cores
* RAM usage
* swap space usage
* information about all running processes (sortable)

Each process in a Linux system has a PID (process ID) which is a unique identifier for the process.  We can use this PID to send signals to the running process through `htop`.  

One common signal we might want to send is for the process to shut itself down.  There are a few different signals we can send to kill a process, but the two most common are `SIGTERM` and `SIGKILL`.  `SIGTERM` is a polite way to ask a process to stop itself and the process is allowed to handle this signal (often used for clean up) or even to ignore the signal completely.  This is the default way most processes are ended.  `SIGKILL` is a more extreme version which ends the process immediately and does not allow the process to handle or ignore it.  If `SIGKILL` fails to kill the process (very rare) it can be a sign of an OS bug.  For more information on these termination signals as well as other, check out [here](https://www.gnu.org/software/libc/manual/html_node/Termination-Signals.html).

Of course, we don't need to use `htop` to end processes, we can also use the `kill` command.  To send a `SIGTERM`, we can run `kill PID` and to send a `SIGKILL`, we can run `kill -KILL PID`.  Also check out the `pkill` command which can sometimes be a bit more convenient than `kill` as it allows you to kill processes by name rather than PID.

## Useful Bash Commands


We've covered some of the most commonly used commands in Bash, but there are many more useful commands and popular applications frequently utilized by Bash users. Among these are `apt` or `apt-get` for package management, `cron` for scheduling tasks, `tmux` for managing the terminal itself, `rsync` for managing large-scale file transfers, `tar` for archiving and compression, and many more. The history of Unix dates back nearly half a century; if there's a task you'd like to do, there's a good chance someone has developed an application for it. Explore the Unix-based/Unix-like software ecosystem!

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*